In [1]:
import re
import os
import glob

In [2]:
fpt = '/home/despoB/lesion/data/original/nifti/sub_{}/anat/t1mprage.nii.gz'

In [3]:
nifti_dir = '/home/despoB/lesion/data/original/nifti'

In [4]:
epi_rgx = re.compile('(EPITemplatephysiomatchedN[A-Z]?\.nii\.gz)|(EPITemplatephysiomatched[A-Z]?\.nii\.gz)')

### Look through files in `../originals/nifti` to determine which patients have full data ready

In [5]:
t1w_scans = []
missing_t1w = []
missing_anat = []
func_old = []
func_new = []
missing_epi = []
no_func_scans = []
missing_func = []
for sub_id in os.listdir(nifti_dir):
    # Define full path to /anat
    anat_path = os.path.join(nifti_dir, sub_id, 'anat')
    # If /anat exists...
    if os.path.isdir(anat_path):
        # Define the path where we should find a t1w scan
        t1wpath = os.path.join(anat_path, 't1mprage.nii.gz')
        # If a t1w scan exists..
        if os.path.isfile(t1wpath):
            # Add to the list of t1w scans
            t1w_scans.append(t1wpath)
        else:
            # Add to the list of patients missing t1w scans
            missing_t1w.append(sub_id)
    else:
        # Add to the list of patients missing /anat
        missing_anat.append(sub_id)
    
    # Define full path to /func
    func_path = os.path.join(nifti_dir, sub_id, 'func')
    # If /func exists...
    if os.path.isdir(func_path):
        # Get the contents of /func
        func_files = os.listdir(func_path)
        # If there are files in /func
        if len(func_files) > 0:
            # Initialize func_counter
            func_counter = 0
            # Loop through the files
            for scan in func_files:
                # Check if the scan is an EPI
                regx_res = epi_rgx.match(scan)
                # If the scan is an EPI..
                if regx_res:
                    # If it is an old sequence...
                    if regx_res.groups()[0] is None:
                        # Add to the list of old scans
                        func_old.append(os.path.join(func_path, scan))
                    else:
                        # Add to the list of new scans
                        func_new.append(os.path.join(func_path, scan))
                    # Increment func_counter
                    func_counter += 1
            # If there were no EPI scans..
            if func_counter == 0:
                missing_epi.append(sub_id)
        # If there were no func scans at all...
        else:
            # Add to list of patients with no func scans.
            no_func_scans.append(sub_id)
    # If /func is missing...
    else:
        # Add to list of patients missing /func
        missing_func.append(sub_id)

out_arrays = [t1w_scans, missing_t1w, missing_anat, func_old, func_new,
                  missing_epi, no_func_scans, missing_func]
for oarray in out_arrays:
    oarray.sort()

In [6]:
missing_t1w_set = set(missing_t1w)
missing_anat_set = set(missing_anat)
missing_epi_set = set(missing_epi)
no_func_scans_set = set(no_func_scans)
missing_func_set = set(missing_func)

In [7]:
not_ready = missing_anat_set | missing_epi_set | missing_func_set | missing_t1w_set | no_func_scans_set

In [8]:
ready_set = set(os.listdir(nifti_dir)) - not_ready

In [9]:
ready_set = set([i[4:7] for i in list(ready_set)])

### Look through `../mask_prep` to see which patients have masks ready

In [10]:
mask_files = os.listdir('/home/despoB/lesion/mask_prep/combined')

In [11]:
has_mask = [i[0:3] for i in mask_files]

In [12]:
has_mask_set = set(has_mask)

In [13]:
ready_with_mask = ready_set & has_mask_set

In [14]:
ready_with_mask = list(ready_with_mask)

In [15]:
ready_with_mask.sort()

### Create BIDS symlink structure for patients whose data and mask are ready

In [17]:
from pathlib import Path

In [18]:
BIDS_DIR = Path('/home/despo/dlurie/Projects/despolab_lesion/first_pass_analysis/input')
ORIG_DIR = Path('/home/despoB/lesion/data/original/nifti')
MASK_DIR = Path('/home/despoB/lesion/mask_prep/combined')
EPI_META = '/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/input/sub-175/func/sub-175_task-rest_run-01_bold.json'

# for each patient in ready_with_mask
for pat_id in ready_with_mask:
    # set the BIDS subject ID
    sub_id = 'sub-'+pat_id
    print(sub_id)
    # set the path to the BIDS subject anat and func directories
    anat_dir = BIDS_DIR.joinpath(sub_id, 'anat')
    func_dir = BIDS_DIR.joinpath(sub_id, 'func')
    # create a subject directory and anat/func subdirectories
    anat_dir.mkdir(parents=True)
    func_dir.mkdir()
    # Set the patient existing ID
    orig_id = 'sub_'+pat_id
    # Set the original data directory
    pat_origs = ORIG_DIR.joinpath(orig_id)
    # create a symlink to the t1w image in BIDS/anat
    t1w_fname = 'sub-{}_T1w.nii.gz'.format(pat_id)
    os.symlink(str(pat_origs.joinpath('anat', 't1mprage.nii.gz')), str(anat_dir.joinpath(t1w_fname)))
    # create a symlink to the mask image in BIDS/anat
    mask_fname = 'sub-{}_t1w_label-lesion_roi.nii.gz'.format(pat_id)
    os.symlink(str(MASK_DIR.joinpath(pat_id+'_manual_mask.nii.gz')), str(anat_dir.joinpath(mask_fname)))
    # check which func protocol this patient has
    old_res = [i for i in func_old if pat_id in i]
    new_res = [i for i in func_new if pat_id in i]
    # initialize run_counter
    run_counter = 1
    # if old protocol..
    if old_res:
        # Sort the list of EPI scans
        old_res.sort()
        # loop through EPI scans
        for epi_scan in old_res:
            # create a symlink and label the EPI scan
            epi_fname = 'sub-{0}_task-rest_run-{1:02d}_bold.nii.gz'.format(pat_id, run_counter)
            os.symlink(epi_scan, str(func_dir.joinpath(epi_fname)))
            # create a symlink to the EPI metaflie
            meta_fname = 'sub-{0}_task-rest_run-{1:02d}_bold.json'.format(pat_id, run_counter)            
            os.symlink(EPI_META, str(func_dir.joinpath(meta_fname)))
            # increment run_counter
            run_counter += 1
    elif new_res:
        # Sort the list of EPI scans
        new_res.sort()
        # loop through EPI scans
        for epi_scan in new_res:
            # create a symlink and label the EPI scan
            epi_fname = 'sub-{0}_task-rest_run-{1:02d}_bold.nii.gz'.format(pat_id, run_counter)
            os.symlink(epi_scan, str(func_dir.joinpath(epi_fname)))
            # create a symlink to the EPI metaflie
            meta_fname = 'sub-{0}_task-rest_run-{1:02d}_bold.json'.format(pat_id, run_counter)            
            os.symlink(EPI_META, str(func_dir.joinpath(meta_fname)))
            # increment run_counter
            run_counter += 1

sub-101
sub-102
sub-103
sub-104
sub-106
sub-151
sub-152
sub-153
sub-154
sub-155
sub-156
sub-157
sub-159
sub-160
sub-161
sub-164
sub-165
sub-166
sub-167
sub-169
sub-170
sub-171
sub-172
sub-174
sub-175
sub-177
sub-178
sub-179
sub-180
sub-181
sub-182
sub-185
sub-191
